In [ ]:
import pandas as pd
import numpy as np
import plotly
import ace_tools_open as tools
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import ace_tools_open as tools

AttributeError: module 'pytorch_lightning' has no attribute 'utilities'

In [8]:
!pip install neuralprophet


In [ ]:
# File paths for all uploaded stock data
file_paths = {
    # Consumer Goods Sector
    "CPIN": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Consumer Goods/CPIN.JK_dataprice.xlsx",
    "ICBP": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Consumer Goods/ICBP.JK_dataprice.xlsx",
    "INDF": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Consumer Goods/INDF.JK_dataprice.xlsx",
    "UNVR": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Consumer Goods/UNVR.JK_dataprice.xlsx",
    
    # Energy and Utilities Sector
    "ADRO": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Energy and Utils/ADRO.JK_dataprice.xlsx",
    "PGAS": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Energy and Utils/PGAS.JK_dataprice.xlsx",
    "PTBA": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Energy and Utils/PTBA.JK_dataprice.xlsx",

    # Finance Sector
    "BBCA": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Finance/BBCA.JK_dataprice.xlsx",
    "BBNI": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Finance/BBNI.JK_dataprice.xlsx",
    "BBTN": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Finance/BBTN.JK_dataprice.xlsx",
    "BMRI": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Finance/BBTN.JK_dataprice.xlsx",
    "BBRI": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Finance/BBRI.JK_dataprice.xlsx",
    
    # Healthcare Sector
    "KLBF": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Healthcare/KLBF.JK_dataprice.xlsx",
    "MIKA": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Healthcare/MIKA.JK_dataprice.xlsx",
    
    
    # Mines and Basic Materials Sector
    "ANTM": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/ANTM.JK_dataprice.xlsx",
    "BRPT": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/BRPT.JK_dataprice.xlsx",
    "INCO": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/INCO.JK_dataprice.xlsx",
    "INKP": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/INKP.JK_dataprice.xlsx",
    "MDKA": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/MDKA.JK_dataprice.xlsx",
    "SMGR": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/SMGR.JK_dataprice.xlsx",
    "TINS": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/TINS.JK_dataprice.xlsx",
    "UNTR": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Mines and Basic Materials/UNTR.JK_dataprice.xlsx",

    # Otomotive and Industrial Sector
    "ASII": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Otomotive and Industrials/ASII.JK_dataprice.xlsx",

    # Tech Sector
    "BUKA": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Tech/BUKA.JK_dataprice.xlsx",
    "EMTK": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Tech/EMTK.JK_dataprice.xlsx",
    "GOTO": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Tech/GOTO.JK_dataprice.xlsx",

    # Telecommunication Sector
    "EXCL": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Telecommunications/EXCL.JK_dataprice.xlsx",
    "TBIG": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Telecommunications/TBIG.JK_dataprice.xlsx",
    "TLKM": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Telecommunications/TLKM.JK_dataprice.xlsx",
    "TOWR": "CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Telecommunications/TOWR.JK_dataprice.xlsx",
}


In [ ]:
# === Clean NeuralProphet Forecast with Plotly ===
# Load and prepare the data
df = pd.read_excel("CS-IDX30 version 3/CS-IDX30 version 3/pricehistory data/Finance/BBCA.JK_dataprice.xlsx", sheet_name="Sheet1")
df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df['y'] = pd.to_numeric(df['y'], errors='coerce')
df = df.dropna()

# Confirm data is clean
assert df['y'].apply(lambda x: isinstance(x, (int, float))).all()

# Create and train the model without checkpointing (fixes PyTorch 2.6 error)
model = NeuralProphet(n_lags=5, n_forecasts=1, epochs=100)
model.fit(df, freq='D', minimal=True, checkpointing=False, continue_training=False)

# Forecast
future = model.make_future_dataframe(df, periods=10, n_historic_predictions=True)
forecast = model.predict(future)

# Plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['y'], mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat1'], mode='lines', name='Forecast (t+1)'))

fig.update_layout(
    title='NeuralProphet Forecast vs Actual',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark'
)
fig.show()

KeyboardInterrupt: 

In [ ]:
# Dictionary to store stock data
stock_data = {}

# Load each file into a dictionary
for stock, path in file_paths.items():
    try:
        if path.endswith('.csv'):
            df = pd.read_csv(path, index_col="Date", parse_dates=True)
        elif path.endswith('.xlsx'):
            df = pd.read_excel(path, index_col="Date", parse_dates=True)
        else:
            print(f"Unsupported file format for {stock}: {path}")
            continue
        stock_data[stock] = df
    except Exception as e:
        print(f"Error loading {stock} from {path}: {e}")

# Print the first few rows of each DataFrame to debug
for stock, df in stock_data.items():
    print(f"Data for {stock}:")
    print(df.head())

# Combine all data into a single DataFrame using the "Close" column
merged_df = pd.DataFrame()
print("Columns:", df.columns.tolist())
print("Data preview:")

for stock, df in stock_data.items():
    if "Close" in df.columns:
        merged_df[stock] = df["Close"]
    else:
        print(f"Warning: 'Close' column not found in {stock}")

# Drop rows with missing values
merged_df.dropna(inplace=True)

# Display merged stock data
tools.display_dataframe_to_user(name="Merged Stock Data", dataframe=merged_df)

Data for CPIN:
                   Open         High          Low        Close    Volume  \
Date                                                                       
2017-07-31  2530.030696  2575.699120  2365.624369  2438.693848  66502600   
2017-08-01  2466.094728  2575.698938  2447.827360  2530.030518  29682500   
2017-08-02  2530.030518  2575.698938  2520.896833  2530.030518  13380800   
2017-08-03  2530.030518  2548.297886  2511.763149  2530.030518   6098000   
2017-08-04  2484.362266  2593.966484  2484.362266  2566.565430   7424900   

            Dividends  Stock Splits  
Date                                 
2017-07-31          0             0  
2017-08-01          0             0  
2017-08-02          0             0  
2017-08-03          0             0  
2017-08-04          0             0  
Data for ICBP:
                   Open         High          Low        Close    Volume  \
Date                                                                       
2017-07-31  7590.25

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense, Dropout

def train_multivariate_lstm(df, stock_name,
                            seq_len=60,
                            epochs=50,
                            batch_size=32,
                            units=50,
                            dropout=0.2,
                            learning_rate=0.001):

    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import mean_squared_error
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from tensorflow.keras.optimizers import Adam

    print(f"\n📈 Training LSTM model for {stock_name}...")

    # 1. Prepare input features
    features = df[['Close', 'Volume', 'Dividends', 'Stock Splits']].copy().dropna()
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(features)

    # 2. Split
    split = int(len(scaled) * 0.8)
    train_data = scaled[:split]
    val_data = scaled[split - seq_len:]

    def create_sequences(data):
        X, y = [], []
        for i in range(seq_len, len(data)):
            X.append(data[i - seq_len:i])
            y.append(data[i, 0])  # 'Close'
        return np.array(X), np.array(y)

    X_train, y_train = create_sequences(train_data)
    X_val, y_val = create_sequences(val_data)

    # 3. Build model
    model = Sequential([
        LSTM(units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(dropout),
        LSTM(units),
        Dropout(dropout),
        Dense(1)
    ])
    opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # 4. Save model
    model.save(f'model_{stock_name}_SL{seq_len}_U{units}_DO{dropout}_BS{batch_size}_LR{learning_rate}.h5')

    # 5. Predict
    y_pred = model.predict(X_val)
    y_pred_full = np.zeros((len(y_pred), 4))
    y_pred_full[:, 0] = y_pred[:, 0]
    y_val_full = np.zeros((len(y_val), 4))
    y_val_full[:, 0] = y_val
    y_pred_real = scaler.inverse_transform(y_pred_full)[:, 0]
    y_val_real = scaler.inverse_transform(y_val_full)[:, 0]

    # 6. RMSE + Plot
    rmse = np.sqrt(mean_squared_error(y_val_real, y_pred_real))
    print(f"📉 RMSE (validation): {rmse:.2f}")
    # plt.figure(figsize=(10, 5))
    # plt.plot(y_val_real, label='Actual Price')
    # plt.plot(y_pred_real, label='Predicted Price')
    # plt.title(f"{stock_name} - Actual vs Predicted (Validation)")
    # plt.xlabel("Days")
    # plt.ylabel("Price (IDR)")
    # plt.legend()
    # plt.show()

    return model, scaler, rmse


In [ ]:
from itertools import product

def tune_lstm_hyperparameters(df, stock_name):
    # Define ranges to test
    seq_lens = [30, 60, 90]
    lstm_units = [50, 100, 150]
    dropout_rates = [0.2, 0.3]
    batch_sizes = [16, 32]
    learning_rates = [0.001, 0.0005]

    results = []

    combinations = list(product(seq_lens, lstm_units, dropout_rates, batch_sizes, learning_rates))

    print(f"🔍 Testing {len(combinations)} combinations...\n")

    for seq_len, units, dropout, batch, lr in combinations:
        print(f"🔧 Trying: seq_len={seq_len}, units={units}, dropout={dropout}, batch_size={batch}, lr={lr}")
        try:
            _, _, rmse = train_multivariate_lstm(
                df=df,
                stock_name=stock_name,
                seq_len=seq_len,
                epochs=30,  
                batch_size=batch,
                units=units,
                dropout=dropout,
                learning_rate=lr
            )
            results.append({
                'seq_len': seq_len,
                'units': units,
                'dropout': dropout,
                'batch_size': batch,
                'learning_rate': lr,
                'rmse': rmse
            })
        except Exception as e:
            print(f"⚠️ Skipped due to error: {e}")

    results.sort(key=lambda x: x['rmse'])

    print("\n🏁 Top 5 Results:")
    for r in results[:5]:
        print(r)

    return results


In [ ]:
BBCA_df = stock_data['BBCA']
results = tune_lstm_hyperparameters(BBCA_df, "BBCA")

🔍 Testing 72 combinations...

🔧 Trying: seq_len=30, units=50, dropout=0.2, batch_size=16, lr=0.001

📈 Training LSTM model for BBCA...


c:\Users\Wesley\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
📉 RMSE (validation): 156.60
🔧 Trying: seq_len=30, units=50, dropout=0.2, batch_size=16, lr=0.0005

📈 Training LSTM model for BBCA...


c:\Users\Wesley\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
📉 RMSE (validation): 233.33
🔧 Trying: seq_len=30, units=50, dropout=0.2, batch_size=32, lr=0.001

📈 Training LSTM model for BBCA...


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step
📉 RMSE (validation): 305.89
🔧 Trying: seq_len=90, units=150, dropout=0.3, batch_size=32, lr=0.001

📈 Training LSTM model for BBCA...


c:\Users\Wesley\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step
📉 RMSE (validation): 437.96
🔧 Trying: seq_len=90, units=150, dropout=0.3, batch_size=32, lr=0.0005

📈 Training LSTM model for BBCA...


c:\Users\Wesley\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
📉 RMSE (validation): 161.74

🏁 Top 5 Results:
{'seq_len': 90, 'units': 150, 'dropout': 0.3, 'batch_size': 16, 'learning_rate': 0.001, 'rmse': np.float64(125.38762397829232)}
{'seq_len': 60, 'units': 100, 'dropout': 0.2, 'batch_size': 16, 'learning_rate': 0.001, 'rmse': np.float64(134.0058746140335)}
{'seq_len': 90, 'units': 150, 'dropout': 0.2, 'batch_size': 16, 'learning_rate': 0.001, 'rmse': np.float64(135.8263024698593)}
{'seq_len': 30, 'units': 100, 'dropout': 0.2, 'batch_size': 16, 'learning_rate': 0.001, 'rmse': np.float64(138.6232193728952)}
{'seq_len': 30, 'units': 150, 'dropout': 0.2, 'batch_size': 32, 'learning_rate': 0.001, 'rmse': np.float64(139.37445386226418)}
